In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
from lightgbm import LGBMClassifier, Booster
from matplotlib import pyplot as plt
from feature_extraction import get_features, get_features_with_derivative

In [11]:
painting_data = [{'name': 'Rothko', 'colors': ['#f8b335', '#ed6a29', '#f39434', '#fdc03e', '#fa3229']},
                 {'name': 'Monet', 'colors': ['#848aa7', '#9392a8', '#6f7ca5', '#b2918a', '#9b8d9c']},
                 {'name': 'Picasso', 'colors': ['#132f3a', '#224f5b', '#93a49c', '#103755', '#436160']},
                 {'name': 'Cuco', 'colors': ['#be8373', '#9f9646', '#8ca487', '#768cb2', '#568132']},
                 {'name': 'Bacon', 'colors': ['#562f4c', '#a42238', '#4a181c', '#ba252c', '#7d212c']}]

n_features = (14 + 3)  # 3 - autocorr, 14 - main
samplerate = 45
max_len_autocorr = 45

In [13]:
data_files = ['EEG_Data/data01_g.txt',
              'EEG_Data/data02_g.txt',
              'EEG_Data/data03_g.txt',
              'EEG_Data/data04_g.txt',
              'EEG_Data/data05_g.txt',
              'EEG_Data/data01.txt',
              'EEG_Data/data02.txt',
              'EEG_Data/data03.txt',
              'EEG_Data/data04.txt',
              'EEG_Data/data05.txt',
              'EEG_Data/data01_d.txt',
              'EEG_Data/data02_d.txt',
              'EEG_Data/data03_d.txt',
              'EEG_Data/data04_d.txt',
              'EEG_Data/data05_d.txt',
              'EEG_Data/data01_a.txt',
              'EEG_Data/data02_a.txt',
              'EEG_Data/data03_a.txt',
              'EEG_Data/data04_a.txt',
              'EEG_Data/data05_a.txt',
              'EEG_Data/data01_k.txt',
              'EEG_Data/data02_k.txt',
              'EEG_Data/data03_k.txt',
              'EEG_Data/data04_k.txt',
              'EEG_Data/data05_k.txt']

labels = [3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2]

train_dfs = [pd.read_csv(f, sep=' ', header=None) for f in data_files]

for tr_df in train_dfs:
    print(tr_df.shape)

(1470, 10)
(1465, 10)
(1470, 10)
(1465, 10)
(1460, 10)
(1455, 10)
(1465, 10)
(1515, 10)
(1465, 10)
(1505, 10)
(1470, 10)
(1480, 10)
(1465, 10)
(1480, 10)
(1450, 10)
(1470, 10)
(1470, 10)
(1475, 10)
(1570, 10)
(1470, 10)
(1510, 10)
(1470, 10)
(1465, 10)
(1465, 10)
(1500, 10)


In [14]:
usecols = list(range(5))
n_channels = len(usecols)

In [15]:
def get_samples_from_pd(pd, arr_len=256, step=64, n_samples=12):
    output = np.zeros((n_samples, n_features * n_channels))
    for i in range(n_samples):
        for j in range(n_channels):
            feat_list = get_features(pd[usecols].values[i * step: i * step+arr_len, j])
            output[i, j*n_features:(j+1)*n_features] = feat_list
    return output

In [22]:
use_samples = 50

In [23]:
Y_full = np.zeros(len(labels) * use_samples)
for i in range(len(labels)):
    Y_full[i*use_samples:(i+1) * use_samples] = labels[i]

In [24]:
train_data = np.zeros((use_samples * len(labels), n_features * n_channels))

for i in range(len(labels)):
    train_data[i*use_samples:(i+1) * use_samples,:] = get_samples_from_pd(train_dfs[i],
                                                                          arr_len=samplerate,
                                                                          step=5, n_samples=use_samples)

/Users/georgeoblapenko/Dropbox/Kunstmord/GODA/Neurotheater/feature_extraction.py:39: RuntimeWarning: divide by zero encountered in double_scalars
  rms, rms / std,
/Users/georgeoblapenko/Dropbox/Kunstmord/GODA/Neurotheater/feature_extraction.py:40: RuntimeWarning: divide by zero encountered in double_scalars
  array[argmax] / array[argmin],


In [25]:
train_data = np.nan_to_num(train_data)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(train_data, Y_full, test_size=0.2)

In [31]:
lgb = LGBMClassifier(num_leaves=10, min_data=5, min_data_in_bin=5, learning_rate=0.01, n_estimators=2000)
lgb.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=100)

[1]	valid_0's multi_logloss: 1.59644
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's multi_logloss: 1.58368
[3]	valid_0's multi_logloss: 1.57123
[4]	valid_0's multi_logloss: 1.55917
[5]	valid_0's multi_logloss: 1.5473
[6]	valid_0's multi_logloss: 1.53573
[7]	valid_0's multi_logloss: 1.52459
[8]	valid_0's multi_logloss: 1.51366
[9]	valid_0's multi_logloss: 1.50304
[10]	valid_0's multi_logloss: 1.49258
[11]	valid_0's multi_logloss: 1.4821
[12]	valid_0's multi_logloss: 1.47127
[13]	valid_0's multi_logloss: 1.4606
[14]	valid_0's multi_logloss: 1.45001
[15]	valid_0's multi_logloss: 1.43982
[16]	valid_0's multi_logloss: 1.42981
[17]	valid_0's multi_logloss: 1.41972
[18]	valid_0's multi_logloss: 1.41021
[19]	valid_0's multi_logloss: 1.39942
[20]	valid_0's multi_logloss: 1.38877
[21]	valid_0's multi_logloss: 1.37855
[22]	valid_0's multi_logloss: 1.36841
[23]	valid_0's multi_logloss: 1.35829
[24]	valid_0's multi_logloss: 1.34837
[25]	valid_0's multi_logloss: 1.33863

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.01, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_data=5, min_data_in_bin=5,
        min_split_gain=0.0, n_estimators=2000, n_jobs=-1, num_leaves=8,
        objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
        silent=True, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=0)

In [30]:
lgb.booster_.save_model('lightGBM_sr_45_1sec_all5_v2.txt')